In [ ]:
using RigidBodyDynamics
using StaticArrays: SVector

In [ ]:
base = RigidBody{Float64}("base")
mechanism = Mechanism(base)

In [ ]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating{Float64}());
joint1ToBase = rand(Transform3D{Float64}, joint1.frameBefore, default_frame(base))
body1ToJoint1 = rand(Transform3D{Float64}, default_frame(body1), joint1.frameAfter)
attach!(mechanism, base, joint1, joint1ToBase, body1, body1ToJoint1);

In [ ]:
joint2 = Joint("joint2", Revolute(rand(SVector{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, default_frame(body1))
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [ ]:
mechanism

In [ ]:
x = MechanismState(Float64, mechanism);

In [ ]:
rand!(x)
setdirty!(x)
configuration(x, joint1)

In [ ]:
num_positions(x)

In [ ]:
num_velocities(x)

In [ ]:
relative_transform(x, joint2.frameAfter, default_frame(body1))

In [ ]:
center_of_mass(x)

In [ ]:
mass(mechanism)

In [ ]:
transform(x, Point3D(Float64, joint2.frameAfter), root_frame(mechanism))

In [ ]:
relative_twist(x, body2, body1)

In [ ]:
relative_twist(x, body1, base) + relative_twist(x, body2, body1) + relative_twist(x, base, body2)

In [ ]:
p = path(mechanism, body2, base)
J = geometric_jacobian(x, p)

In [ ]:
Twist(J, velocity_vector(x, p))

In [ ]:
mass_matrix(x)

In [ ]:
A = momentum_matrix(x)

In [ ]:
Momentum(A, velocity_vector(x))

In [ ]:
accel = SpatialAcceleration(joint2.frameAfter, default_frame(base), joint2.frameAfter, rand(SVector{3, Float64}), rand(SVector{3, Float64}))
transform(x, accel, default_frame(base))

In [ ]:
bias_acceleration(x, body2)

In [ ]:
v̇ = rand(num_velocities(mechanism))
τ = inverse_dynamics(x, v̇)

In [ ]:
relative_acceleration(x, body1, body2, v̇)

In [ ]:
# random chain
rand_chain_mechanism(Float64, QuaternionFloating{Float64}, Revolute{Float64}, Revolute{Float64})

In [ ]:
# random tree
rand_tree_mechanism(Float64, [QuaternionFloating{Float64}; [Revolute{Float64} for i = 1 : 10]]...)